In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

%matplotlib inline



In [2]:
train = pd.read_csv('~/Octokitty/Personal/Picklehack-2020/proof_of_concept/data/train.csv')
test = pd.read_csv('~/Octokitty/Personal/Picklehack-2020/proof_of_concept/data/test.csv')

In [3]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [9]:
# Removing Twitter handles

combi = train.append(test, ignore_index=True)

def remove_pattern(input_txt, pattern):
  r = re.findall(pattern, input_txt)
  for i in r:
    input_txt = re.sub(i, '', input_txt)

  return input_txt

combi['tidy_tweet'] = np.vectorize(remove_pattern)(combi['tweet'], "@[\w]*")


In [10]:
# Remove special characters, numbers, punctuation

combi['tidy_tweet'] = combi['tidy_tweet'].str.replace("[^a-zA-Z#]", " " )

In [11]:
# Removing short words

combi['tidy_tweet'] = combi['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [12]:
combi.head()

,id,label,tweet,tidy_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,when father dysfunctional selfish drags kids i...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks #lyft credit cause they offer wheelchai...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model love take with time
4,5,0.0,factsguide: society now #motivation,factsguide society #motivation


In [13]:
tokenized_tweet = combi['tidy_tweet'].apply(lambda x: x.split())

tokenized_tweet.head()

0    [when, father, dysfunctional, selfish, drags, ...
1    [thanks, #lyft, credit, cause, they, offer, wh...
2                              [bihday, your, majesty]
3                     [#model, love, take, with, time]
4                   [factsguide, society, #motivation]
Name: tidy_tweet, dtype: object

In [14]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x])

In [15]:
tokenized_tweet.head()

0    [when, father, dysfunct, selfish, drag, kid, i...
1    [thank, #lyft, credit, caus, they, offer, whee...
2                              [bihday, your, majesti]
3                     [#model, love, take, with, time]
4                         [factsguid, societi, #motiv]
Name: tidy_tweet, dtype: object

In [16]:
for i in range(len(tokenized_tweet)):
  tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

combi['tidy_tweet'] = tokenized_tweet

In [13]:
combi.head()

,id,label,tweet,tidy_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,when father dysfunct selfish drag kid into dys...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thank #lyft credit caus they offer wheelchair ...
2,3,0.0,bihday your majesty,bihday your majesti
3,4,0.0,#model i love u take with u all the time in ...,#model love take with time
4,5,0.0,factsguide: society now #motivation,factsguid societi #motiv


In [15]:
def hashtag_extract(x):
  hashtags = []
  for i in x:
    ht = re.findall(r"#(\w+)", i)
    hashtags.append(ht)

  return hashtags

In [16]:
HT_regular = hashtag_extract(combi['tidy_tweet'][combi['label'] == 0])

HT_negative = hashtag_extract(combi['tidy_tweet'][combi['label'] == 1])

HT_regular = sum(HT_regular, [])
HT_negative = sum(HT_negative, [])

In [17]:
# Bag of words model
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')

bow = bow_vectorizer.fit_transform(combi['tidy_tweet'])

In [18]:
#TDIDF model
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(combi['tidy_tweet'])

`poop!`

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [22]:
train_bow = bow[:31962, :]
test_bow = bow[31962:, :]

Now use train_test_split() to split data into "things", need to do this on input data

In [50]:
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow, train['label'], random_state=42, test_size=0.3)

LogisticRegression method, and fit data

In [51]:
lreg = LogisticRegression()
lreg.fit(xtrain_bow, ytrain) 

LogisticRegression()

In [52]:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

LogisticRegression(multi_class='warn', solver='warn')

save lreg var to file!

In [57]:
prediction = lreg.predict_proba(xvalid_bow) # input
print(xvalid_bow)
print(len(prediction), prediction[:,1])
prediction_int = prediction[:,1] >= 0.3
prediction_int = prediction_int.astype(np.int) # Classifications
print(prediction_int)

f1_score(yvalid, prediction_int)

  (0, 791)	2
  (0, 389)	1
  (0, 746)	2
  (0, 843)	2
  (1, 210)	1
  (1, 544)	1
  (1, 495)	1
  (1, 930)	1
  (2, 393)	1
  (2, 614)	1
  (2, 881)	1
  (2, 929)	1
  (2, 902)	1
  (2, 38)	1
  (2, 451)	1
  (3, 336)	1
  (3, 881)	1
  (3, 237)	1
  (4, 302)	1
  (4, 210)	1
  (4, 882)	1
  (4, 303)	1
  (4, 359)	1
  (5, 590)	2
  (5, 338)	1
  :	:
  (9582, 220)	1
  (9583, 23)	1
  (9583, 509)	1
  (9583, 168)	1
  (9583, 222)	1
  (9583, 911)	1
  (9583, 245)	2
  (9584, 557)	1
  (9584, 518)	1
  (9584, 888)	1
  (9585, 988)	1
  (9585, 922)	1
  (9585, 593)	1
  (9586, 632)	2
  (9586, 963)	1
  (9586, 7)	1
  (9586, 887)	1
  (9587, 135)	1
  (9587, 388)	1
  (9587, 719)	1
  (9587, 702)	1
  (9587, 78)	1
  (9588, 729)	1
  (9588, 228)	1
  (9588, 285)	1
9589 [0.00074547 0.0111488  0.01521938 ... 0.0590835  0.67134737 0.02090845]
[0 0 0 ... 0 1 0]


0.5303408146300915

LogisticRegression with 

In [29]:
train_tfidf = tfidf[:31962, :]
test_tfidf = tfidf[31962:, :]

In [30]:
xtrain_tfidf = train_tfidf[ytrain.index]
xvalid_tfidf = train_tfidf[yvalid.index]

In [31]:
lreg.fit(xtrain_tfidf, ytrain)

LogisticRegression()

In [42]:
prediction = lreg.predict_proba(xvalid_tfidf)
prediction_int = prediction[:, 1] >= 0.3
prediction_int = prediction_int.astype(np.int)

f1_score(yvalid, prediction_int)

0.4555903866248694

In [48]:
combi.head()

,id,label,tweet,tidy_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,when father dysfunct selfish drag kid into dys...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thank #lyft credit caus they offer wheelchair ...
2,3,0.0,bihday your majesty,bihday your majesti
3,4,0.0,#model i love u take with u all the time in ...,#model love take with time
4,5,0.0,factsguide: society now #motivation,factsguid societi #motiv


In [49]:
print(prediction)

[[9.99254526e-01 7.45474291e-04]
 [9.88851197e-01 1.11488035e-02]
 [9.84780617e-01 1.52193834e-02]
 ...
 [9.40916501e-01 5.90834991e-02]
 [3.28652633e-01 6.71347367e-01]
 [9.79091545e-01 2.09084547e-02]]
